In [ ]:
# Install required packages
!pip install numpy pandas scikit-learn nltk scikit-surprise

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [45 lines of output]
      Compiling surprise/similarities.pyx because it changed.
      Compiling surprise/prediction_algorithms/matrix_factorization.pyx because it changed.
      Compiling surprise/prediction_algorithms/optimize_baselines.pyx because it changed.
      Compiling surprise/prediction_algorithms/slope_one.pyx because it changed.
      Compiling surprise/prediction_algorithms/co_clustering.pyx because it changed.
      [1/5] Cythonizing surprise/prediction_algorithms/co_clustering.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
              self.avg_cltr_i = avg_cltr_i
              self.avg_cocltr = avg_cocltr
      
              return self
      
          def compute_averages(self, np.ndarray[np.int_t] cltr_u,
                                                   ^
  


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


In [3]:
# Download dataset
# If you already have the dataset, you can skip or comment this step
!kaggle datasets download -d hsankesara/medium-articles

In [26]:
# Import and Load Data
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('dataset/articles.csv')
columns = ['author', 'claps', 'reading_time', 'link', 'title', 'text']
df = df[columns]
df = df.dropna()

df['claps'] = pd.to_numeric(df['claps'], errors='coerce').fillna(0).astype(int)
print(f"Dataset loaded with {len(df)} articles")

Dataset loaded with 337 articles


In [18]:
# Data Preparation
available_columns = ['author', 'claps', 'reading_time', 'link', 'title', 'text']  
df = df[available_columns]
df = df.dropna()
print("Dataset shape:", df.shape)

Dataset shape: (337, 6)


In [19]:
# Recommendation Function
def get_recommendations(topic, n=2):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['text'])
    
    query_vec = tfidf.transform([topic])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    
    # Get top matches and sort by claps
    indices = similarity.argsort()[-n*2:][::-1]  
    candidates = df.iloc[indices]
    return candidates.nlargest(n, 'claps')  

In [20]:
# Interactive Interface
def ask_for_recommendations():
    topic = input("What topic are you interested in? ")
    print(f"\nFinding articles about '{topic}'...\n")
    
    try:
        recommendations = get_recommendations(topic)
        print("Recommended Articles:")
        for _, article in recommendations.iterrows():
            print(f"\nTitle: {article['title']}")
            print(f"Author: {article['author']}")
            print(f"Reading time: {article['reading_time']} mins")
            print(f"Claps: {article['claps']}")
            print(f"Link: {article['link']}")
            print("-" * 50)
    except Exception as e:
        print(f"Error: {str(e)}")

In [24]:
# Run System
ask_for_recommendations()


Finding articles about 'data'...

Error: Column 'claps' has dtype object, cannot use method 'nlargest' with this dtype
